In [1]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent.parent))

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("Petitions.csv")
df

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство
...,...,...,...
59884,3128111,прошу закрасить граффити,Благоустройство
59885,3276713,Прошу вас отремонтировать пешеходную дорожку,Благоустройство
59886,3274663,Необходимо демонтировать незаконную рекламную ...,Незаконная информационная и (или) рекламная ко...
59887,3359308,Очень гремит на ветру металлическая часть окна...,Кровля


In [4]:
from mylib.preprocessing.preprocessing import preprocessing

In [5]:
from nltk.corpus import stopwords
import string


sw = stopwords.words('russian')
sw.extend(list(string.punctuation))

In [6]:
sub_expressions = {
    '<[^>]*>': "",
    "&(.*);": "",
    "/": " ",
    "[^\s^\w^-]+": "",
    " -* ": " ",
    " \d+ ": " ",
}

In [7]:
dataset = list(map(lambda s: 
                   preprocessing(s, ["apply_regular_expressions", "tokenize", "lemmatization", "drop_stopwords"], 
                                 sub_expressions=sub_expressions, tokenization_type='word', stopwords=sw), list(df['public_petition_text'])))

In [8]:
text_corpus = [word for text_document in dataset for word in text_document if len(word) > 1 and word.isalpha()]

In [9]:
vocabulary = set(text_corpus)

In [10]:
id_word = {id: word for id, word in enumerate(vocabulary)}

In [11]:
word_id = {word: id for id, word in id_word.items()}

In [12]:
text_corpus_id = list(pd.DataFrame(text_corpus)[0].map(word_id))

In [13]:
WINDOW = 2

In [14]:
from mylib.dataset.create_context_dataset import create_context_dataset

context_dataset_id = create_context_dataset(text_corpus_id, WINDOW)

In [15]:
context_dataset_id

[(9101, [18957, 8202, 9688, 4968]),
 (18957, [8202, 3265, 9101, 9688]),
 (8202, [3265, 6574, 18957, 9101]),
 (3265, [6574, 17529, 8202, 18957]),
 (6574, [17529, 4697, 3265, 8202]),
 (17529, [4697, 19191, 6574, 3265]),
 (4697, [19191, 11496, 17529, 6574]),
 (19191, [11496, 17563, 4697, 17529]),
 (11496, [17563, 5213, 19191, 4697]),
 (17563, [5213, 2043, 11496, 19191]),
 (5213, [2043, 5873, 17563, 11496]),
 (2043, [5873, 6913, 5213, 17563]),
 (5873, [6913, 19001, 2043, 5213]),
 (6913, [19001, 7192, 5873, 2043]),
 (19001, [7192, 18933, 6913, 5873]),
 (7192, [18933, 18933, 19001, 6913]),
 (18933, [18933, 13596, 7192, 19001]),
 (18933, [13596, 13137, 18933, 7192]),
 (13596, [13137, 6142, 18933, 18933]),
 (13137, [6142, 3175, 13596, 18933]),
 (6142, [3175, 12804, 13137, 13596]),
 (3175, [12804, 12206, 6142, 13137]),
 (12804, [12206, 7981, 3175, 6142]),
 (12206, [7981, 10361, 12804, 3175]),
 (7981, [10361, 10175, 12206, 12804]),
 (10361, [10175, 4467, 7981, 12206]),
 (10175, [4467, 16386, 103

# 1 Матрица контекстных эмбеддингов

In [16]:
from mylib.word2vec.create_contextual_embeddings_matrix import create_contextual_embeddings_matrix
import numpy as np


contextual_embeddings_matrix = create_contextual_embeddings_matrix(context_dataset_id, len(vocabulary))

In [17]:
np.sum(contextual_embeddings_matrix, axis=1)

array([  4., 192.,   8., ..., 216.,   8., 692.])

In [18]:
from sklearn.decomposition import PCA

contextual_embeddings_matrix_pca = PCA(3).fit_transform(contextual_embeddings_matrix)

In [19]:
np.savetxt('contextual_embeddings_matrix_pca.tsv', contextual_embeddings_matrix_pca, delimiter='\t')

In [20]:
d = pd.DataFrame.from_dict(id_word, orient='index')
d.to_csv("id_word.tsv", sep='\t')

# 2 CBoW

In [21]:
import torch.nn as nn
import torch


class CBoW(nn.Module):
    def __init__(self, vocabulary_size: int, embedding_size: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.first_linear = nn.Linear(embedding_size, 300)
        self.ReLU = nn.ReLU()
        self.second_linear = nn.Linear(300, vocabulary_size) 

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = self.embedding(x).mean(1).squeeze(1) # усреднение по контексту batch_size x 2 * window x embedding_size -> batch_size x embedding_size
        y = self.first_linear(y)
        y = self.ReLU(y)
        y = self.second_linear(y)
        return y

In [22]:
EMBEDDING_SIZE = 150

In [23]:
model = CBoW(len(vocabulary), EMBEDDING_SIZE)

In [24]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)

In [25]:
from torch.utils.data import TensorDataset, DataLoader


train_ds = TensorDataset(torch.tensor([elem[1] for elem in context_dataset_id]), torch.tensor([elem[0] for elem in context_dataset_id]))
train_dl = DataLoader(train_ds, batch_size=300)

In [26]:
epochs = 10
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        outputs = model(x_b)
        loss_value = loss(outputs, y_b)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 6.196937561035156
Эпоха 2, Значение функции потерь: 5.408342361450195
Эпоха 3, Значение функции потерь: 4.762289047241211
Эпоха 4, Значение функции потерь: 4.17322301864624
Эпоха 5, Значение функции потерь: 3.7651360034942627
Эпоха 6, Значение функции потерь: 3.417372465133667
Эпоха 7, Значение функции потерь: 3.154480218887329
Эпоха 8, Значение функции потерь: 2.9273781776428223
Эпоха 9, Значение функции потерь: 2.7420239448547363
Эпоха 10, Значение функции потерь: 2.5853140354156494


In [27]:
cbow_embeddings_matrix = model.second_linear.weight.detach().numpy()

In [28]:
cbow_embeddings_matrix_pca = PCA(3).fit_transform(cbow_embeddings_matrix)

In [29]:
np.savetxt('cbow_embeddings_matrix_pca.tsv', cbow_embeddings_matrix_pca, delimiter='\t')

# 3 Skip Gram

In [43]:
class SkipGram(nn.Module):
    def __init__(self, vocabulary_size: int, embedding_size: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.first_linear = nn.Linear(embedding_size, 300)
        self.ReLU = nn.ReLU()
        self.second_linear = nn.Linear(300, vocabulary_size) 

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = self.embedding(x)
        y = self.first_linear(y)
        y = self.ReLU(y)
        y = self.second_linear(y)
        return y

In [48]:
model_sg = SkipGram(len(vocabulary), EMBEDDING_SIZE)

In [49]:
loss_sg = nn.CrossEntropyLoss()
optimizer_sg = torch.optim.Adam(model_sg.parameters(), lr=0.0025)

In [58]:
train_ds_sg = TensorDataset(torch.tensor([elem[0] for elem in context_dataset_id[:1000]]), torch.tensor([elem[1] for elem in context_dataset_id[:1000]]))
train_dl_sg = DataLoader(train_ds_sg, batch_size=100)

In [60]:
epochs = 20
for epoch in range(epochs):
    for x_b, y in train_dl_sg:
        for y_b in y.T:
            outputs = model_sg(x_b)
            loss_value = loss_sg(outputs, y_b)
            optimizer_sg.zero_grad()
            loss_value.backward()
            optimizer_sg.step()
        
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 2.669610023498535
Эпоха 2, Значение функции потерь: 2.575890302658081
Эпоха 3, Значение функции потерь: 2.549670934677124
Эпоха 4, Значение функции потерь: 2.5297718048095703
Эпоха 5, Значение функции потерь: 2.48268985748291
Эпоха 6, Значение функции потерь: 2.4909780025482178
Эпоха 7, Значение функции потерь: 2.4415531158447266
Эпоха 8, Значение функции потерь: 2.4656729698181152
Эпоха 9, Значение функции потерь: 2.434032440185547
Эпоха 10, Значение функции потерь: 2.446873664855957
Эпоха 11, Значение функции потерь: 2.4132351875305176
Эпоха 12, Значение функции потерь: 2.4307923316955566
Эпоха 13, Значение функции потерь: 2.3882811069488525
Эпоха 14, Значение функции потерь: 2.4193499088287354
Эпоха 15, Значение функции потерь: 2.399970293045044
Эпоха 16, Значение функции потерь: 2.408480644226074
Эпоха 17, Значение функции потерь: 2.3859469890594482
Эпоха 18, Значение функции потерь: 2.395367383956909
Эпоха 19, Значение функции потерь: 2.3767173290

In [61]:
sg_embeddings_matrix = model_sg.second_linear.weight.detach().numpy()

In [62]:
sg_embeddings_matrix_pca = PCA(3).fit_transform(sg_embeddings_matrix)

In [ ]:
np.savetxt('sg_embeddings_matrix_pca.tsv', sg_embeddings_matrix_pca, delimiter='\t')